In [1]:
from CharRNN import CharRNN
import torch, torch.optim as optim, torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from onehotencoder import onehotencoder
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
#Hyperparameters
vocab_size = 28
embedded_dim = 200
hidden_dim = 768
num_layers = 3
dropout = .4
learning_rate = 1e-3
num_epochs = 100
batch_size = 64
molecule_size = 200
temp = .4
p = .9

In [3]:
#Basic one hot encoder i made to encode and decode both characters and sequences
endecode = onehotencoder()

#Torch dataset because the proccessed inputs and outputs were over 60gb in size
class SequenceDataset(Dataset):
    def __init__(self, file_path, encoder):
        self.file_path = file_path
        self.encoder = encoder
        with open(file_path, 'r') as f:
            self.lines = f.readlines()

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        sequence = self.lines[idx].strip()
        input_tensor = self.encoder.encode_sequence(sequence)
        target_tensor = self.encoder.encode_sequence(sequence, targets=True)
        return input_tensor, target_tensor

#Load the dataset for working
dataset = SequenceDataset('data/train.csv', endecode)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers= 3)

In [ ]:
#Declare RNN with vocab size, padding index, embedded dim size, hidden dim size, drop out %
charRNN = CharRNN(
    vocab_size,
    torch.argmax(endecode.encode('[PAD]')).item(),
    embedded_dim,
    hidden_dim,
    num_layers,
    dropout,
).to(device)

#Using basic cross entropy loss
criterion = nn.CrossEntropyLoss()

#AdamW
optimizer = optim.AdamW(charRNN.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3)

#Typical training loop
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer.zero_grad()
        
        logits = charRNN(batch_inputs)

        targets_flat = torch.argmax(batch_targets, dim=2).reshape(-1)
        logits_flat = logits.reshape(batch_inputs.size(0) * batch_inputs.size(1), -1)
        
        loss = criterion(logits_flat, targets_flat)
        loss.backward()
        optimizer.step()
    scheduler.step(loss.item())
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

KeyboardInterrupt: 

In [ ]:
#This is a bit wonky as its turning the output into a probability distribution and then takes the smallest group of logits to add up to the probability of top_p then samples those
def top_p_filtering(logits, top_p, temp):
    probs = nn.functional.softmax(logits.squeeze(0)[-1] / temp, dim=0)
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=0) 
    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1].clone()
    sorted_indices_to_remove[0] = False
    indices_to_remove = sorted_indices_to_remove.scatter(0, sorted_indices, sorted_indices_to_remove)
    filtered_probs = probs.masked_fill(indices_to_remove, 0).clone()
    filtered_probs = filtered_probs / filtered_probs.sum()
    next_token_index = torch.multinomial(filtered_probs, 1).item()
    return next_token_index

In [ ]:
#Inputs the start token then does top p sampling until it generates the stop token[EOS] to it hits 200 characters whatever comes first.
currenToken = endecode.encode('[BOS]').to(device)
charRNN.eval()
generation = []
with torch.no_grad():
    for i in range(200):
        if currenToken.dim() == 2:
            currenToken = currenToken.unsqueeze(0)
        logits = charRNN(currenToken)
        next_token_index = top_p_filtering(logits, p, temp)
        next_token = torch.zeros(vocab_size)
        next_token[next_token_index] = 1
        char = endecode.decode(next_token)
        if next_token_index == endecode.encode('[EOS]').argmax().item():
            break
        generation.append(char)
        currenToken = next_token.unsqueeze(0).unsqueeze(0).to(device)

print(''.join(generation))

In [ ]:
torch.save(charRNN,'Models/charRNN.pt')